# Merging Allocs/Accesses Files from iti.files
Taking multipe files from v20-testing folder and merging them together

Needed to run some extra code to assure columns match up correctly with their values and to remove extra column. Also had to remove filler 14 from alloc_ip column.

In [2]:
# imports here
import pandas as pd
import numpy as np
import networkx as nx
import mariadb
import sys
import os
import glob

In [2]:
#files = glob.glob('*.accesses')

#combineddf = pd.DataFrame()

#for file in files:
#    df = pd.read_csv(file)
#    accesses = pd.concat([combineddf,df])

#accesses

In [3]:
#files = glob.glob('*.allocs')

#combined_df = pd.DataFrame()

#for file in files:
#    df2 = pd.read_csv(file)
#    allocs = pd.concat([combined_df,df2])

#allocs = allocs.reset_index()
#allocs = allocs.rename(columns={"index": "alloc_ip", "alloc_ip": "pid", "pid": "obj_ptr", "ptr": "obj_size", "size": "alloc_jiffies", "alloc_time": "free_jiffies","free_time": "free_ip", "free_ip": "type", "type": "command", "slab": "slab", "new_type": "new_type"})
#allocs = allocs.drop(allocs[allocs['alloc_ip'] == '14              '].index)
#allocs = allocs.drop(allocs[allocs['alloc_ip'] == '19              '].index)
#allocs


In [4]:
#duplicates = allocs.drop_duplicates(subset = ['alloc_ip','pid','obj_ptr','obj_size','alloc_jiffies','free_jiffies','free_ip','type','command','slab'], keep = 'first')
#print('Number of Rows After Dropping Duplicates:',len(duplicates))
#duplicates

In [5]:
#df = pd.merge(allocs, accesses, on='alloc_jiffies')
#df = df.drop(columns = ['new_type'])
#df

In [3]:
if __name__ == "__main__":
    try:
        conn = mariadb.connect(
            host=os.getenv("MYSQL_HOST"),
            port=3306,
            user=os.getenv("MYSQL_USER"),
            database=os.getenv("MYSQL_DB"),
            password=os.getenv("MYSQL_PASS"))
    except mariadb.Error as e:
        print(f"Error connecting to the databse: {e}")
        sys.exit(1)

    cursor = conn.cursor()

    # allocs
    files = glob.glob('*.allocs')
    combined_df = pd.DataFrame()

    for file in files:
        df2 = pd.read_csv(file)
        allocs = pd.concat([combined_df,df2])

    allocs = allocs.reset_index()
    allocs = allocs.rename(columns={"index": "alloc_ip", "alloc_ip": "pid", "pid": "obj_ptr", "ptr": "obj_size", "size": "alloc_jiffies", "alloc_time": "free_jiffies","free_time": "free_ip", "free_ip": "type", "type": "command", "slab": "slab", "new_type": "new_type"})
    allocs = allocs.drop(allocs[allocs['alloc_ip'] == '14              '].index)
    allocs = allocs.drop(allocs[allocs['alloc_ip'] == '19              '].index)


    # accesses
    files = glob.glob('*.accesses')
    combineddf = pd.DataFrame()

    for file in files:
        df = pd.read_csv(file)
        accesses = pd.concat([combineddf,df])

    df = pd.merge(allocs, accesses, on='alloc_jiffies')
    df = df.drop(columns=["new_type"])

    insert_query = "INSERT INTO memorizer (alloc_ip, pid, obj_ptr, obj_size, alloc_jiffies, free_jiffies, free_ip, type, command, slab,access_ip,writes,rds) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"

    batch_size = 1000
    for start in range(0, len(df), batch_size):
        batch = df.iloc[start:start+batch_size].values.tolist()
        cursor.executemany(insert_query, batch)
        conn.commit()
        print(f"Inserted rows {start} to {start + len(batch) - 1}")

    cursor.close()
    conn.close()

Error connecting to the databse: Can't connect to server on '10.0.68.10' (10061)


AttributeError: 'tuple' object has no attribute 'tb_frame'